## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-14-20-53-38 +0000,startribune,Live: U.S. men’s hockey team trails Denmark 2-...,https://www.startribune.com/2026-winter-olympi...
1,2026-02-14-20-50-07 +0000,nyt,"Ebo Taylor, Musical Innovator of Highlife and ...",https://www.nytimes.com/2026/02/13/arts/music/...
2,2026-02-14-20-48-00 +0000,bbc,No arrests made after police search residence ...,https://www.bbc.com/news/articles/cly8j19rql7o...
3,2026-02-14-20-47-25 +0000,wapo,Authorities conduct operation near Nancy Guthr...,https://www.washingtonpost.com/nation/2026/02/...
4,2026-02-14-20-46-10 +0000,nyt,"Live Updates: In Munich, Rubio Calls Europe a ...",https://www.nytimes.com/live/2026/02/14/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2493/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
79,trump,32
85,ice,16
174,new,12
598,epstein,11
275,was,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
73,2026-02-14-15-01-39 +0000,nypost,Marco Rubio calls on Europe to join new world ...,https://nypost.com/2026/02/14/world-news/marco...,79
234,2026-02-13-22-19-13 +0000,latimes,Newsom tells world leaders Trump's retreat on ...,https://www.latimes.com/california/story/2026-...,64
247,2026-02-13-21-46-07 +0000,nypost,Trump says regime change in Iran would be the ...,https://nypost.com/2026/02/13/us-news/trump-re...,63
197,2026-02-13-23-36-59 +0000,nyt,Trump Says Regime Change Would Be the ‘Best Th...,https://www.nytimes.com/2026/02/13/us/politics...,61
163,2026-02-14-02-40-00 +0000,cbc,Trump says regime change 'seems' to be 'best t...,https://www.cbc.ca/news/world/iran-us-trump-re...,61


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
73,79,2026-02-14-15-01-39 +0000,nypost,Marco Rubio calls on Europe to join new world ...,https://nypost.com/2026/02/14/world-news/marco...
56,49,2026-02-14-16-29-44 +0000,nypost,Pam Bondi was mocked for calling out crime in...,https://nypost.com/2026/02/14/us-news/man-dead...
238,46,2026-02-13-22-07-58 +0000,nypost,White House unleashes at Gavin Newsom’s Europe...,https://nypost.com/2026/02/13/us-news/newsom-c...
182,46,2026-02-14-00-22-58 +0000,cbc,'Canadians are with you. We will always be wit...,https://www.cbc.ca/news/canada/british-columbi...
101,44,2026-02-14-12-00-00 +0000,startribune,Minneapolis has 950 surveillance cameras. Some...,https://www.startribune.com/minneapolis-survei...
50,42,2026-02-14-17-00-14 +0000,bbc,Russia killed opposition leader Alexei Navalny...,https://www.bbc.com/news/articles/cwyk4lz4e3eo...
72,39,2026-02-14-15-08-20 +0000,nypost,Savannah Guthrie’s husband Michael Feldman see...,https://nypost.com/2026/02/14/us-news/savannah...
158,38,2026-02-14-03-45-00 +0000,wsj,Casey Wasserman Is Putting His Talent Agency U...,https://www.wsj.com/business/casey-wasserman-p...
49,37,2026-02-14-17-02-08 +0000,nypost,200K protesters fight Iran’s Islamic Republic ...,https://nypost.com/2026/02/14/world-news/200k-...
193,32,2026-02-13-23-45-00 +0000,wsj,Anthropic’s artificial-intelligence tool Claud...,https://www.wsj.com/politics/national-security...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
